# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I
output_data_file = pd.read_csv("../WeatherPy/output_data/cities.csv")
output_data_file.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Davila,20,PH,1607443368,94,18.47,120.58,77.00,2.24
1,Atuona,2,PF,1607443240,74,-9.80,-139.03,78.49,15.12
2,New Norfolk,86,AU,1607443286,75,-42.78,147.06,59.00,1.99
3,Zaysan,0,KZ,1607443368,84,47.47,84.87,6.30,1.05
4,Tasiilaq,40,GL,1607443255,57,65.61,-37.64,19.40,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = output_data_file[["Lat", "Lng"]]

# Store humidity in humidity
humidity = output_data_file["Humidity"]

In [4]:
# Plot Heatmap
m = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
m.add_layer(heatmap_layer)

#Display figure (m)
m

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
output_data_file

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Davila,20,PH,1607443368,94,18.47,120.58,77.00,2.24
1,Atuona,2,PF,1607443240,74,-9.80,-139.03,78.49,15.12
2,New Norfolk,86,AU,1607443286,75,-42.78,147.06,59.00,1.99
3,Zaysan,0,KZ,1607443368,84,47.47,84.87,6.30,1.05
4,Tasiilaq,40,GL,1607443255,57,65.61,-37.64,19.40,4.70
...,...,...,...,...,...,...,...,...,...
539,Port Said,0,EG,1607443308,52,31.26,32.28,69.80,8.05
540,Aborlan,40,PH,1607443427,88,9.44,118.55,80.60,4.70
541,Āsind,19,IN,1607443428,35,25.73,74.33,66.63,2.33
542,Dāhānu,0,IN,1607443226,55,19.97,72.73,75.67,6.91


In [6]:
# Narrow down the cities with wind speed less than 20 mph, max_temp between 20 and 30 with humidity of less than 80%
narrowed_city_df = output_data_file.loc[(output_data_file['Wind Speed'] < 20) & (output_data_file['Max Temp'] <= 30) & (output_data_file['Humidity'] <= 65)]
(output_data_file['Max Temp'] <= 30).dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,Tasiilaq,40,GL,1607443255,57,65.61,-37.64,19.40,4.70
39,Albany,96,US,1607443196,61,42.60,-73.97,28.00,3.00
87,Nikolskoye,0,RU,1607443255,62,59.70,30.79,21.99,6.71
90,Asău,100,RO,1607443280,23,46.43,26.40,27.00,1.92
223,Prigorodka,0,RU,1607443392,58,52.04,39.71,24.80,8.95
324,Sept-Îles,26,CA,1607443225,63,50.20,-66.38,28.00,6.93
429,Korfovskiy,0,RU,1607443415,61,48.22,135.06,12.20,11.18
479,Saýat,0,TM,1607443420,57,38.78,63.88,23.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the dataframe
hotel_df["Hotel"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel
4,Tasiilaq,GL,65.61,-37.64,
39,Albany,US,42.60,-73.97,
87,Nikolskoye,RU,59.70,30.79,
90,Asău,RO,46.43,26.40,
223,Prigorodka,RU,52.04,39.71,
324,Sept-Îles,CA,50.20,-66.38,
429,Korfovskiy,RU,48.22,135.06,
479,Saýat,TM,38.78,63.88,


In [8]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dictionary
params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

In [9]:
# Start for loop
for index, row in hotel_df.iterrows():
    # get city name, lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response ['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    # if there is not hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
        
        # Wait 1 second to make another api request to avoid SSL error
        time.sleep(1)

# Print end of search once searching is completed
print("----------End of Search----------")

Retrieving Results for Index 4: Tasiilaq.
Closest hotel in Tasiilaq is The Red House.
Retrieving Results for Index 39: Albany.
Missing field/result...skipping.
Retrieving Results for Index 87: Nikolskoye.
Missing field/result...skipping.
Retrieving Results for Index 90: Asău.
Closest hotel in Asău is Pensiunea Sophia.
Retrieving Results for Index 223: Prigorodka.
Closest hotel in Prigorodka is Bazilik Usman'.
Retrieving Results for Index 324: Sept-Îles.
Closest hotel in Sept-Îles is Château Arnaud.
Retrieving Results for Index 429: Korfovskiy.
Missing field/result...skipping.
Retrieving Results for Index 479: Saýat.
Missing field/result...skipping.
----------End of Search----------


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel,Hotel Name
4,Tasiilaq,GL,65.61,-37.64,,The Red House
39,Albany,US,42.60,-73.97,,NaN
87,Nikolskoye,RU,59.70,30.79,,NaN
90,Asău,RO,46.43,26.40,,Pensiunea Sophia
223,Prigorodka,RU,52.04,39.71,,Bazilik Usman'
324,Sept-Îles,CA,50.20,-66.38,,Château Arnaud
429,Korfovskiy,RU,48.22,135.06,,NaN
479,Saýat,TM,38.78,63.88,,NaN


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Add the layer to the map
m.add_layer(markers)

# Display figure
m

Figure(layout=FigureLayout(height='420px'))